In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dawid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
# initialize paths for data
all_beauty_path = "../data/raw/All_Beauty.jsonl"
handmade_path = "../data/raw/Handmade_Products.jsonl"
health_care_path = "../data/raw/Health_and_Personal_Care.jsonl"

In [30]:
all_beauty_df = pd.read_json(all_beauty_path, lines=True)
handmade_df = pd.read_json(handmade_path, lines=True)
health_care_df = pd.read_json(health_care_path, lines=True)

# ------------- Clean Data -------------

### 1. Delete all reviews with NO verified purchases

In [31]:
all_beauty_df = all_beauty_df[all_beauty_df["verified_purchase"] == True].copy()
all_beauty_df["verified_purchase"].value_counts()

verified_purchase
True    634969
Name: count, dtype: int64

In [32]:
handmade_df = handmade_df[handmade_df["verified_purchase"] == True].copy()
handmade_df["verified_purchase"].value_counts()

verified_purchase
True    632033
Name: count, dtype: int64

In [33]:
health_care_df = health_care_df[health_care_df["verified_purchase"] == True].copy()
health_care_df["verified_purchase"].value_counts()

verified_purchase
True    445072
Name: count, dtype: int64

### 2. Delete all reviews with text length < 10

In [34]:
all_beauty_df = all_beauty_df[all_beauty_df["text"].str.len() > 10].copy()
all_beauty_df.shape

(603846, 10)

In [35]:
handmade_df = handmade_df[handmade_df["text"].str.len() > 10].copy()
handmade_df.shape

(603367, 10)

In [36]:
health_care_df = health_care_df[health_care_df["text"].str.len() > 10].copy()
health_care_df.shape

(424936, 10)

### 3. Delete all unnecessary columns, the only left are: rating, title and text

In [ ]:
# images, asin, parent_asin, user_id, timestamp, helpful_vote, verified_purchase